In [1]:
from neo import NeoGraph
from db import Db

import pandas as pd

g = NeoGraph()

db = Db()
eng = db.create_engine()
df = pd.read_sql("""select * from hsk5k""", eng)

In [2]:
df['len'] = df['characters'].apply(lambda x: len(list(x)))
df['c1'] = df['characters'].apply(lambda x: list(x)[0])
df['c2'] = df['characters'].apply(lambda x: list(x)[1] if len(x) >= 2 else None)
df['p1'] = df['pinyin'].apply(lambda x: x.split(' ')[0])
df['p2'] = df['pinyin'].apply(lambda x: x.split(' ')[1] if ' ' in x else None)
df['english'] = df['english'].apply(lambda x: x.replace("'", "\\'"))
df['english_short'] = df['english'].apply(lambda x: x.split(';')[0])
df['descr'] = df.apply(lambda x: f"""{x['pinyin']} | {x['english_short']}""",axis = 1)

In [3]:
df

,hsk,characters,pinyin,english,len,c1,c2,p1,p2,english_short,descr
0,3,阿姨,a1 yi2,maternal aunt; step-mother; childcare worker; ...,2,阿,姨,a1,yi2,maternal aunt,a1 yi2 | maternal aunt
1,3,啊,a5,"modal particle ending sentence, showing affirm...",1,啊,None,a5,None,"modal particle ending sentence, showing affirm...","a5 | modal particle ending sentence, showing a..."
2,5,唉,ai1,interjection or grunt of agreement or recognit...,1,唉,None,ai1,None,interjection or grunt of agreement or recognit...,ai1 | interjection or grunt of agreement or re...
3,6,哎哟,ai1 yo1,hey; ow; ouch; interjection of pain or surprise,2,哎,哟,ai1,yo1,hey,ai1 yo1 | hey
4,6,挨,ai2,to suffer from; to endure; to tide over (a dif...,1,挨,None,ai2,None,to suffer from,ai2 | to suffer from
...,...,...,...,...,...,...,...,...,...,...,...
4997,6,作息,zuo4 xi1,work and rest,2,作,息,zuo4,xi1,work and rest,zuo4 xi1 | work and rest
4998,3,作业,zuo4 ye4,school assignment; homework; work; task; opera...,2,作,业,zuo4,ye4,school assignment,zuo4 ye4 | school assignment
4999,3,作用,zuo4 yong4,to act on; to affect; action; function; activi...,2,作,用,zuo4,yong4,to act on,zuo4 yong4 | to act on
5000,6,座右铭,zuo4 you4 ming2,motto; maxim,3,座,右,zuo4,you4,motto,zuo4 you4 ming2 | motto


In [4]:
df.groupby('c1').size().sort_values(ascending=False)[:10]

c1
不    37
一    25
发    25
公    23
无    21
打    21
对    19
反    18
人    18
大    17
dtype: int64

In [5]:
df.groupby('c2').size().sort_values(ascending=False)[:10]

c2
子    53
心    32
气    25
理    24
力    23
动    22
然    21
不    21
定    19
实    19
dtype: int64

In [6]:
helper = []
chars = []
for i, c in df.iterrows():
    for h, p in zip(list(c['characters']), c['pinyin'].split(' ')):
        helper.append(f'{h} : {p}')
        chars.append(h)
        

In [7]:
from collections import Counter, OrderedDict

In [8]:
g.truncate()

----- Truncating graph -----


In [9]:
g.add_characters(list(set(chars)), df)

  0%|          | 13/2634 [00:00<00:20, 129.40it/s]

----- Adding chars -----


100%|██████████| 2634/2634 [00:07<00:00, 362.39it/s]


----- Done -----


In [10]:
g.create_links(df[df['len'] == 2])

  0%|          | 13/4025 [00:00<00:31, 127.31it/s]

----- Linking two character words -----


100%|██████████| 4025/4025 [00:19<00:00, 206.33it/s]

----- Done -----
